In [4]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 595 kB/s 
     |████████████████████████████████| 6.4 MB 11.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import math
import yfinance as yf
import requests as rq
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [6]:
aapl = yf.Ticker("AAPL")

# get stock info
aapl.info

# get historical market data
hist = aapl.history(period="48mo")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-07-02,44.180308,45.016709,44.084168,44.987865,70925200,0.0,0.0
2018-07-03,45.134470,45.172926,44.113002,44.204334,55819200,0.0,0.0
2018-07-05,44.526399,44.802798,44.290861,44.560047,66416800,0.0,0.0
2018-07-06,44.564845,45.288284,44.511969,45.177727,69940800,0.0,0.0
2018-07-09,45.545469,45.829075,45.497400,45.805042,79026400,0.0,0.0
...,...,...,...,...,...,...,...
2022-06-27,142.699997,143.490005,140.970001,141.660004,70207900,0.0,0.0
2022-06-28,142.130005,143.419998,137.320007,137.440002,67083400,0.0,0.0
2022-06-29,137.460007,140.669998,136.669998,139.229996,66242400,0.0,0.0


In [7]:
#Get the shape of the rows and columns in the dataset
hist.shape

(1008, 7)